In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys
sys.path.append("/home/149/ab8992/tasman-tides/")
import ttidelib as tt
import xrft 
import os
import shutil
from dask.distributed import Client
# client = Client()
# client
os.chdir("/home/149/ab8992/tasman-tides/rundirs/blank-20-synbath")

## Troubleshoot postprocessing

In [ ]:
from pathlib import Path
import numpy as np
import os
import shutil
import xarray as xr
from dask.distributed import Client
import haversine
from pathlib import Path
import sys
sys.path.append("/home/149/ab8992/tasman-tides/")
import ttidelib as tt
import json
import argparse

#TODO Allow python file to take arguments for chunking?



# parser = argparse.ArgumentParser()
# parser.add_argument('-p', '--to_process', type=str, help='outputs to process. If 0, runs all, if -1 runs only last')
# parser.add_argument('-c', '--yb-chunksize', type=str, help='size of chunks across beam',default=12)
# args = parser.parse_args()

# to_process = args.to_process
# yb_chunksize = args.yb_chunksize
#! Temporary
to_process = 8
yb_chunksize = 12
os.chdir("/home/149/ab8992/tasman-tides/rundirs/blank-20-synbath")
#! Temporary

hourly_diags = {
    "rho":
    {"x":"xh","y":"yh","z":"z_i"},
    "khh":
    {"x":"xh","y":"yh","z":"z_l"},
    "e":
    {"x":"xh","y":"yh","z":"rho2_i"}
}

daily_diags = [
    "KE_stress"
    "KE_visc"
    "KE_horvisc"
    "PE_to_KE"
    "dKE_dt"
]

def save_chunked(data,name,chunks = yb_chunksize):
    if not (gdataout / f"{name}").exists():
        (gdataout / f"{name}").mkdir(parents=True)
    i = 0
    while i * chunks < data["yb"].shape[0]:
        data.isel(
            {
                "yb" : slice(i*10,(i+1)*10)
                }
                ).to_netcdf(gdataout / f"{name}" / f"{name}_y{i:02d}.nc")
        i += 1


if __name__ == "__main__":
    client = Client()
    print(client)
    rundir = Path.cwd()
    # Get the name of folder from Path object
    expt = rundir.name
    print(f"Running postprocessing for experiment {expt}")

    if to_process == "-1":
        # Find most recent output folder
        i = 0
        while (rundir / f"archive/output{i:03d}").exists():
            i += 1
        i -=1
        outputs = [f"output{i:03d}"]

    if to_process == "0":
        # Find all output folders
        i = 0
        outputs = []
        while (rundir / f"archive/output{i:03d}").exists():
            outputs.append(f"output{i:03d}")
            i += 1
    else:
        outputs = [f"output{int(to_process):03d}"]

    for output in outputs:
        print(f"\t\t Processing {output}")
        # Set up the run and output directories
        mom6out = rundir /  f"archive/{output}"
        print(f"Processing {mom6out}")
        gdataout = Path("/g/data/nm03/ab8992/outputs") / expt / f"{output}"
        if not gdataout.exists():
            gdataout.mkdir(parents=True)

        ## Simply move the surface variables to gdata. These are unchunked and for the entire domain

        try:
            surface_filename = list(mom6out.glob('*.surface.nc'))[0].name
            shutil.copy(str(mom6out / surface_filename),str(gdataout / "surface.nc"))
        except Exception as e:
            print("Couldn't move surface.nc")
            print(e)

        # Now we do the biggest ones, the hourly diagnostics. These are output in their own folder, chunked along y dimension
        # First do the velocities together, as these need to be summed along and against the beam

        theta = np.arctan((-43.3 + 49.8) / -17) # This is the angle of rotation
        u = xr.open_mfdataset(
            str(mom6out / f"*u.nc"),
            chunks={"zl": 10,"time":10},
            decode_times=False,
        ).sel(xq = slice(145,170),yh = slice(-55,-40)).u
        v = xr.open_mfdataset(
            str(mom6out / f"*v.nc"),
            chunks={"zl": 10,"time":10},
            decode_times=False,
        ).sel(xh = slice(145,170),yq = slice(-55,-40)).v

        u = tt.beamgrid(u,xname = "xq",chunks = yb_chunksize)
        v = tt.beamgrid(v,yname = "yq",chunks = yb_chunksize)

        # Rotate the velocities
        u_rot = u * np.cos(theta) - v * np.sin(theta)
        v_rot = u * np.sin(theta) + v * np.cos(theta)

        # Set the name of u to "u" and description to "velocity along beam"
        u_rot.name = "u"
        u_rot.attrs["long_name"] = "Velocity along beam (Eastward positive)"
        v_rot.name = "v"
        v_rot.attrs["long_name"] = "Velocity across beam (Northward positive)"

        save_chunked(u_rot,"u",chunks = yb_chunksize)
        save_chunked(v_rot,"v",chunks = yb_chunksize)

        ## Now do the rest of the hourly diagnostics
        for diag in hourly_diags:
            print(f"processing {diag}")
            try:
                ds = xr.open_mfdataset(
                    str(mom6out / f"*{diag}.nc"),
                    chunks={hourly_diags[diag]["z"]: 10,"time":10},
                    decode_times=False,
                )[diag].sel({hourly_diags[diag]["x"] : slice(145,170), hourly_diags[diag]["y"] : slice(-55,-40)})
            except Exception as e:
                print(f"Failed to open {diag}")
                print(e)
                continue

            out = tt.beamgrid(ds,xname = hourly_diags[diag]["x"],yname = hourly_diags[diag]["y"])
            out = out.chunk({"yb": yb_chunksize}).persist()

            save_chunked(out,diag,chunks = yb_chunksize)


/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42851 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:42097' processes=7 threads=28, memory=112.00 GiB>
Running postprocessing for experiment blank-20-synbath
		 Processing output008
Processing /home/149/ab8992/tasman-tides/rundirs/blank-20-synbath/archive/output008


/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xesmf/frontend.py:696: UserWarning: Using dimensions ('lat', 'lon') from data variable u as the horizontal dimensions for the regridding.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xesmf/frontend.py:696: UserWarning: Using dimensions ('lat', 'lon') from data variable v as the horizontal dimensions for the regridding.
  warnings.warn(


2023-11-18 18:19:59,877 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('sum-d27aedc3be0f85b1d77ab6abf9b7221d', 22, 2, 2, 0, 0, 0))" coro=<Worker.execute() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/distributed/worker_state_machine.py:3610>> ended with CancelledError
2023-11-18 18:19:59,894 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('open_dataset-u-getitem-936269e461237fe9b987ce31c923be2c', 6, 3, 2, 2))" coro=<Worker.execute() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/distributed/worker_state_machine.py:3610>> ended with CancelledError
2023-11-18 18:19:59,894 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name='gather_dep(tcp://127.0.0.1:35663, {...})' coro=<Worker.gather_dep() done, defined at /g/data/hh5/public/apps/minico

## Ttide lib

In [1]:
from matplotlib import rc
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import os
import haversine
import xarray as xr
import subprocess
import matplotlib.pyplot as plt
import shutil
import xesmf
import pathlib as Path


def xy_to_lonlat(x,y,x0,y0):
    """
    All outputs are in degrees
    """
    lat = np.arcsin((y + y0) / 6371)
    lon = np.arcsin((x + x0) / (6371 * np.cos(lat)))
    #! HARDCODED FOR QUADRANT 2
    lon = np.pi - lon
    return lon * 180 / np.pi,lat * 180 / np.pi

def lonlat_to_xy(lon,lat,lon0,lat0):
    """
    All inputs are in degrees
    """
    R = 6371
    lon /= 180 / np.pi
    lat /= 180 / np.pi
    lon0 /= 180 / np.pi
    lat0 /= 180 / np.pi
    x0,y0 = R * np.cos(lat0) * np.sin(lon0) , R * np.sin(lat0)
    x,y = R * np.cos(lat) * np.sin(lon) - x0, R * np.sin(lat) - y0

    return x,y

def beamgrid(data,lat0 = -42.1,lon0 = 147.2,beamwidth = 400,beamlength = 1500,plot = False,xname = "xh",yname = "yh",vmin = None,vmax = None,chunks = 12):
    # make a docstring describing these variables
    """
    data : xarray.DataArray
        The data to be gridded
    lat0 : float
        Latitude of the origin of the beam
    lon0 : float
        Longitude of the origin of the beam
    beamwidth : float
        Width of the beam in km
    beamlength : float
        Length of the beam in km
    res : float
        Resolution of the grid in km
    plot : bool
        Whether to plot the grid. If plotting, only pass dataarray
    chunks : int
        Chunk size for dask along the yb axis
    Return a xarray.DataArray cut down to size on to the beam grid. The resolution is automatically determined from the base grid.

    """

    if plot == True:
        assert isinstance(data,xr.DataArray), "Data must be an xarray.DataArray"

    lon = data[xname].data
    lat = data[yname].data 

    theta = np.arctan((-43.3 + 49.8) / -17) #! Hardcoded. This comes out to -20.9 degrees
    theta *= -1 ## Look, I just did some trial and error until the beam was in the right quadrant. Who needs year 10 maths
    res = haversine.haversine((lat[0],lon[0]),(lat[0],lon[1]))
    res = np.ceil(res)
    LAT , LON = np.meshgrid(lat,lon)

    ## Define target grid on rotated mesh in km
    y_ = np.linspace(
        -0.5 * beamwidth,
        0.5 * beamwidth,
        int(beamwidth // res) + 1)
    x_ = np.linspace(
        0,
        -1 * beamlength,
        int(beamlength // res) + 1)
    
    X_,Y_ = np.meshgrid(x_,y_)
    ## Define the rotated grid as represented on the original grid. I.E, the points on x_ y_ as represented on the x,y coordinate system
    theta_ = theta
    Xrot , Yrot = X_ * np.cos(theta_) - Y_ * np.sin(theta_) , X_ * np.sin(theta_) + Y_ * np.cos(theta_)
    ## Calculate X,Y (the rotated grid points) as lat/lon
    x0,y0 = 6371 * np.cos(lat0 * np.pi/180) * np.sin(lon0 * np.pi/180) , 6371 * np.sin(lat0 * np.pi/180)
    LONrot,LATrot = xy_to_lonlat(Xrot,Yrot,x0,y0)

    ## Create target grid to interpolate onto
    print("making newgrid")
    newgrid = xr.DataArray(
        data = X_ * 0,
        dims = ["yb","xb"],
        coords = {
            "xb":(["xb"], - X_[0,:]), ## This sets the coordinate as running from Tasmania -> Mac ridge
            "yb":(["yb"],Y_[:,0]),
            "lon":(["yb","xb"],LONrot),
            "lat":(["yb","xb"],LATrot),
        }
    )

    print("making new grid")
    regridder = xesmf.Regridder(
    data.isel(z_l = 0,time = 0),newgrid,"bilinear"
    )
    print("regridding")
    out = regridder(
        data.rename({xname:"lon",yname:"lat"}),keep_attrs = True
        )
    # assign attributes to out
    out.attrs = data.attrs
    out.attrs["Description"] = f"Beamwidth {beamwidth}km, Beamlength {beamlength}km, Resolution {res}km, angle {theta} degrees, origin {lat0,lon0}"
    out.xb.attrs = {
            "long_name": "Distance along beam from Tasmania towards Macquarie Ridge",
            "units": "km",
    }
    out.yb.attrs = {
            "long_name": "Distance perpendicular to beam referened from beam centre",
            "units": "km",
    }
    out.lon.attrs = {
            "long_name": "Longitude of grid point",
            "units": "degrees",
    }
    out.lat.attrs = {
            "long_name": "Latitude of grid point",
            "units": "degrees",
    }

    if "z_l" in out.dims:
        out = out.rename({"z_l":"zl"})
    if "z_i" in out.dims:
        out = out.rename({"z_i":"zi"})
    if plot == False:
        # return out.chunk({"yb": chunks}).persist()
        return out


In [2]:
from pathlib import Path
import numpy as np
import os
import shutil
import xarray as xr
from dask.distributed import Client
import haversine
from pathlib import Path
import sys
sys.path.append("/home/149/ab8992/tasman-tides/")
import ttidelib as tt
import json
#TODO Have postprocessing script read the diag table to figure out which variables to process?

#TODO Allow python file to take arguments for chunking?

hourly_diags = {
    "rho":
    {"x":"xh","y":"yh","z":"z_l"},
    "khh":
    {"x":"xh","y":"yh","z":"z_l"},
    "e":
    {"x":"xh","y":"yh","z":"rho2_i"}
}

daily_diags = [
    "KE_stress"
    "KE_visc"
    "KE_horvisc"
    "PE_to_KE"
    "dKE_dt"
]

def save_chunked(data,name,chunks = 12):
    if not (gdataout / f"{name}").exists():
        (gdataout / f"{name}").mkdir(parents=True)
    i = 0
    while i * chunks < data["yb"].shape[0]:
        data.isel(
            {
                "yb" : slice(i*10,(i+1)*10)
                }
                ).to_netcdf(gdataout / f"{name}" / f"{name}_y{i:02d}.nc")
        i += 1




#! Temporary
os.chdir("/home/149/ab8992/tasman-tides/rundirs/blank-20-synbath")
#! Remporarly

if __name__ == "__main__":
    client = Client()
    print(client)
    rundir = Path.cwd()
    # Get the name of folder from Path object
    expt = rundir.name
    print(f"Running postprocessing for experiment {expt}")
    # Find most recent output folder
    i = 0
    while (rundir / f"archive/output{i:03d}").exists():
        i += 1
    i -=1
    output = f"output{i:03d}"

    # #! Temporaray
    output = "output008"
    # #! Temporary

    # Set up the run and output directories
    mom6out = rundir /  f"archive/{output}"
    print(f"Processing {mom6out}")
    gdataout = Path("/g/data/nm03/ab8992/outputs") / expt / f"{output}"
    if not gdataout.exists():
        gdataout.mkdir(parents=True)

    ## Simply move the surface variables to gdata. These are unchunked and for the entire domain

    try:
        surface_filename = list(mom6out.glob('*.surface.nc'))[0].name
        shutil.copy(str(mom6out / surface_filename),str(gdataout / "surface.nc"))
    except Exception as e:
        print("Couldn't move surface.nc")
        print(e)

    # Now we do the biggest ones, the hourly diagnostics. These are output in their own folder, chunked along y dimension
    # First do the velocities together, as these need to be summed along and against the beam

    theta = np.arctan((-43.3 + 49.8) / -17) # This is the angle of rotation
    u = xr.open_mfdataset(
        str(mom6out / f"*u.nc"),
        chunks={"z_l": 10,"time":10,"xq":-1,"yh":-1},
        decode_times=False,
    ).u.sel(xq = slice(145,170),yh = slice(-55,-40))
    v = xr.open_mfdataset(
        str(mom6out / f"*v.nc"),
        chunks={"z_l": 10,"time":10,"xh":-1,"yq":-1},
        decode_times=False,
    ).v.sel(xh = slice(145,170),yq = slice(-55,-40))
    u = beamgrid(u,xname = "xq",chunks = 12).persist()
    v = beamgrid(v,yname = "yq",chunks = 12).persist()

    ## Maybe have only u THEN v in memory at once?

    # Rotate the velocities
    u_rot = u * np.cos(theta) - v * np.sin(theta)
    v_rot = u * np.sin(theta) + v * np.cos(theta)

    # Set the name of u to "u" and description to "velocity along beam"
    u_rot.name = "u"
    u_rot.attrs["long_name"] = "Velocity along beam (Eastward positive)"
    v_rot.name = "v"
    v_rot.attrs["long_name"] = "Velocity across beam (Northward positive)"

    save_chunked(u_rot,"u",chunks = 12)
    save_chunked(v_rot,"v",chunks = 12)
    del u
    del v
    del u_rot
    del v_rot
    ## Now do the rest of the hourly diagnostics
    for diag in hourly_diags:
        print(f"processing {diag}")
        try:
            ds = xr.open_mfdataset(
                str(mom6out / f"*{diag}.nc"),
                chunks={hourly_diags[diag]["z"]: 10,"time":10,"xh":-1,"yq":-1},
                decode_times=False,
            )[diag].sel({hourly_diags[diag]["x"] : slice(145,170), hourly_diags[diag]["y"] : slice(-55,-40)})
        except Exception as e:
            print(f"Failed to open {diag}")
            print(e)
            continue

        out = tt.beamgrid(ds,xname = hourly_diags[diag]["x"],yname = hourly_diags[diag]["y"]).persist()

        save_chunked(out,diag,chunks = 12)
        del out

<Client: 'tcp://127.0.0.1:42353' processes=7 threads=28, memory=112.00 GiB>
Running postprocessing for experiment blank-20-synbath
Processing /home/149/ab8992/tasman-tides/rundirs/blank-20-synbath/archive/output008


/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "z_l" starting at index 10. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


making newgrid
making new grid
regridding


/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xesmf/frontend.py:696: UserWarning: Using dimensions ('lat', 'lon') from data variable u as the horizontal dimensions for the regridding.
  warnings.warn(


making newgrid
making new grid
regridding


...

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "z_l" starting at index 10. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xesmf/frontend.py:696: UserWarning: Using dimensions ('lat', 'lon') from data variable rho as the horizontal dimensions for the regridding.
  warnings.warn(


processing khh


/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "z_l" starting at index 10. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xesmf/frontend.py:696: UserWarning: Using dimensions ('lat', 'lon') from data variable khh as the horizontal dimensions for the regridding.
  warnings.warn(


processing e


/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "rho2_i" starting at index 10. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 10. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xesmf/frontend.py:696: UserWarning: Using dimensions ('lat', 'lon') from data variable e as the horizontal dimensions for the regridding.
  warnings.warn(


In [3]:
u

<xarray.DataArray 'u' (time: 360, z_l: 100, yh: 398, xq: 501)>
dask.array<getitem, shape=(360, 100, 398, 501), dtype=float32, chunksize=(10, 10, 398, 501), chunktype=numpy.ndarray>
Coordinates:
  * xq       (xq) float64 145.0 145.1 145.1 145.2 ... 169.8 169.9 169.9 170.0
  * yh       (yh) float64 -55.0 -54.96 -54.92 -54.89 ... -40.09 -40.06 -40.02
  * z_l      (z_l) float64 2.704 8.126 13.58 ... 5.236e+03 5.339e+03 5.443e+03
  * time     (time) float64 2.133e+05 2.133e+05 ... 2.136e+05 2.136e+05
Attributes:
    units:          m s-1
    long_name:      Zonal velocity
    cell_methods:   z_l:mean yh:mean xq:point time: point
    interp_method:  none

In [3]:
u

<xarray.DataArray 'u' (time: 360, z_l: 100, yh: 398, xq: 501)>
dask.array<getitem, shape=(360, 100, 398, 501), dtype=float32, chunksize=(10, 25, 199, 191), chunktype=numpy.ndarray>
Coordinates:
  * xq       (xq) float64 145.0 145.1 145.1 145.2 ... 169.8 169.9 169.9 170.0
  * yh       (yh) float64 -55.0 -54.96 -54.92 -54.89 ... -40.09 -40.06 -40.02
  * z_l      (z_l) float64 2.704 8.126 13.58 ... 5.236e+03 5.339e+03 5.443e+03
  * time     (time) float64 2.133e+05 2.133e+05 ... 2.136e+05 2.136e+05
Attributes:
    units:          m s-1
    long_name:      Zonal velocity
    cell_methods:   z_l:mean yh:mean xq:point time: point
    interp_method:  none

In [4]:
ub = beamgrid(u,xname = "xq",chunks = 12)
ub

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xesmf/frontend.py:696: UserWarning: Using dimensions ('lat', 'lon') from data variable u as the horizontal dimensions for the regridding.
  warnings.warn(


<xarray.DataArray 'u' (time: 360, zl: 100, yb: 101, xb: 376)>
dask.array<astype, shape=(360, 100, 101, 376), dtype=float32, chunksize=(10, 25, 101, 191), chunktype=numpy.ndarray>
Coordinates:
  * zl       (zl) float64 2.704 8.126 13.58 ... 5.236e+03 5.339e+03 5.443e+03
  * time     (time) float64 2.133e+05 2.133e+05 ... 2.136e+05 2.136e+05
  * xb       (xb) float64 -0.0 4.0 8.0 12.0 ... 1.492e+03 1.496e+03 1.5e+03
  * yb       (yb) float64 -200.0 -196.0 -192.0 -188.0 ... 192.0 196.0 200.0
    lon      (yb, xb) float64 144.7 144.7 144.8 144.8 ... 165.5 165.6 165.6
    lat      (yb, xb) float64 -44.41 -44.42 -44.44 ... -46.45 -46.47 -46.48
Attributes:
    units:          m s-1
    long_name:      Zonal velocity
    cell_methods:   z_l:mean yh:mean xq:point time: point
    interp_method:  none
    Description:    Beamwidth 400km, Beamlength 1500km, Resolution 4.0km, ang...

In [3]:
u

<xarray.DataArray 'u' (time: 360, z_l: 100, yh: 398, xq: 501)>
dask.array<getitem, shape=(360, 100, 398, 501), dtype=float32, chunksize=(10, 25, 199, 191), chunktype=numpy.ndarray>
Coordinates:
  * xq       (xq) float64 145.0 145.1 145.1 145.2 ... 169.8 169.9 169.9 170.0
  * yh       (yh) float64 -55.0 -54.96 -54.92 -54.89 ... -40.09 -40.06 -40.02
  * z_l      (z_l) float64 2.704 8.126 13.58 ... 5.236e+03 5.339e+03 5.443e+03
  * time     (time) float64 2.133e+05 2.133e+05 ... 2.136e+05 2.136e+05
Attributes:
    units:          m s-1
    long_name:      Zonal velocity
    cell_methods:   z_l:mean yh:mean xq:point time: point
    interp_method:  none

## New Nov 21 still running slow??



In [ ]:
from pathlib import Path
import numpy as np
import os
import shutil
import xarray as xr
from dask.distributed import Client
import haversine
from pathlib import Path
import sys
sys.path.append("/home/149/ab8992/tasman-tides/")
import ttidelib as tt
import json
import argparse

#TODO Allow python file to take arguments for chunking?



# parser = argparse.ArgumentParser()
# parser.add_argument('-p', '--to_process', type=str, help='outputs to process. If 0, runs all, if -1 runs only last',default = "-1")
# parser.add_argument('-c', '--yb-chunksize', type=str, help='size of chunks across beam',default=12)
# args = parser.parse_args()

# to_process = args.to_process
# yb_chunksize = args.yb_chunksize

#! Temporaray
yb_chunksize = 12
to_process = 4
output = "output008"
os.chdir("/home/149/ab8992/tasman-tides/rundirs/full-20")
#! Temporary


hourly_diags = {
    "rho":
    {"x":"xh","y":"yh","z":"z_l"},
    "khh":
    {"x":"xh","y":"yh","z":"z_l"},
    "e":
    {"x":"xh","y":"yh","z":"rho2_i"}
}

daily_diags = [
    "KE_stress"
    "KE_visc"
    "KE_horvisc"
    "PE_to_KE"
    "dKE_dt"
]

def save_chunked(data,name,chunks = yb_chunksize):
    if not (gdataout / f"{name}").exists():
        (gdataout / f"{name}").mkdir(parents=True)
    i = 0
    while i * chunks < data["yb"].shape[0]:
        data.isel(
            {
                "yb" : slice(i*10,(i+1)*10)
                }
                ).to_netcdf(gdataout / f"{name}" / f"{name}_y{i:02d}.nc")
        i += 1


if __name__ == "__main__":
    client = Client(threads_per_worker = 2)
    print(client)
    rundir = Path.cwd()
    # Get the name of folder from Path object
    expt = rundir.name
    print(f"Running postprocessing for experiment {expt}")

    if to_process == "-1":
        # Find most recent output folder
        i = 0
        while (rundir / f"archive/output{i:03d}").exists():
            i += 1
        i -=1
        outputs = [f"output{i:03d}"]
        print(f"Processing last output ({outputs[0]})")


    elif to_process == "0":
        print("Processing all outputs...")
        # Find all output folders
        i = 0
        outputs = []
        while (rundir / f"archive/output{i:03d}").exists():
            outputs.append(f"output{i:03d}")
            i += 1

        print(outputs)
    else:
        outputs = [f"output{int(to_process):03d}"]

    for output in outputs:
        print(f"\t\t Processing {output}")
        # Set up the run and output directories
        mom6out = rundir /  f"archive/{output}"
        print(f"Processing {mom6out}")
        gdataout = Path("/g/data/nm03/ab8992/outputs") / expt / f"{output}"
        if not gdataout.exists():
            gdataout.mkdir(parents=True)

        ## Simply move the surface variables to gdata. These are unchunked and for the entire domain

        try:
            surface_filename = list(mom6out.glob('*.surface.nc'))[0].name
            shutil.copy(str(mom6out / surface_filename),str(gdataout / "surface.nc"))
        except Exception as e:
            print("Couldn't move surface.nc")
            print(e)

        # Now we do the biggest ones, the hourly diagnostics. These are output in their own folder, chunked along y dimension
        # First do the velocities together, as these need to be summed along and against the beam

        theta = np.arctan((-43.3 + 49.8) / -17) # This is the angle of rotation
        u = xr.open_mfdataset(
            str(mom6out / f"*u.nc"),
            chunks={"z_l": 10,"time":10,"xq":-1,"yh":-1},
            decode_times=False,
        ).u.sel(xq = slice(144,170),yh = slice(-55,-40))
        v = xr.open_mfdataset(
            str(mom6out / f"*v.nc"),
            chunks={"z_l": 10,"time":10,"xh":-1,"yq":-1},
            decode_times=False,
        ).v.sel(xh = slice(144,170),yq = slice(-55,-40))

        u = tt.beamgrid(u,xname = "xq",chunks = yb_chunksize).persist()
        v = tt.beamgrid(v,yname = "yq",chunks = yb_chunksize).persist()

        # Rotate the velocities
        u_rot = u * np.cos(theta) - v * np.sin(theta)
        v_rot = u * np.sin(theta) + v * np.cos(theta)

        # Set the name of u to "u" and description to "velocity along beam"
        u_rot.name = "u"
        u_rot.attrs["long_name"] = "Velocity along beam (Eastward positive)"
        v_rot.name = "v"
        v_rot.attrs["long_name"] = "Velocity across beam (Northward positive)"

        save_chunked(u_rot,"u",chunks = yb_chunksize)
        save_chunked(v_rot,"v",chunks = yb_chunksize)

        del u
        del v
        del u_rot
        del v_rot
        ## Now do the rest of the hourly diagnostics
        for diag in hourly_diags:
            print(f"processing {diag}")
            try:
                ds = xr.open_mfdataset(
                    str(mom6out / f"*{diag}.nc"),
                    chunks={hourly_diags[diag]["z"]: 10,"time":10,"xh":-1,"yh":-1},
                    decode_times=False,
                )[diag].sel({hourly_diags[diag]["x"] : slice(144,170), hourly_diags[diag]["y"] : slice(-55,-40)})
            except Exception as e:
                print(f"Failed to open {diag}")
                print(e)
                continue

            out = tt.beamgrid(ds,xname = hourly_diags[diag]["x"],yname = hourly_diags[diag]["y"]).persist()

            save_chunked(out,diag,chunks = yb_chunksize)
            del out

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37961 instead
  warnings.warn(
2023-11-21 15:52:23,217 - distributed.nanny - ERROR - Failed to initialize worker
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/distributed/nanny.py", line 947, in run
    worker = worker_factory()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/distributed/worker.py", line 720, in __init__
    ServerNode.__init__(
TypeError: Server.__init__() got an unexpected keyword argument 'nworkers'
2023-11-21 15:52:23,217 - distributed.nanny - ERROR - Failed to initialize worker
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/distributed/

2023-11-21 15:54:45,204 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:39495'.
2023-11-21 15:54:45,204 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/distributed/comm/tcp.py", line 224, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/distributed/worker.py", line 1255, in heartbeat
    response = await retry_operation(
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/distributed/utils_comm.py", line 455, in retry_operation
    return await retry(
  File "/g/data/hh5/pub